In [2]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5103)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [3]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [4]:
path = get_file('nietzche.txt', 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read()
print("Corpus length: %s" % len(text))

Corpus length: 600901


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
chars.insert(0, "\0")

In [6]:
print("Total chars: ", vocab_size)

Total chars:  86


In [7]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [8]:
char_indices = dict((c, i) for (i, c) in enumerate(chars))
indices_char = dict((i, c) for (i, c) in enumerate(chars))

In [9]:
idx = [char_indices[c] for c in text]

In [10]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [11]:
''.join([chars[i] for i in idx[:100]])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not ground\nfor suspecting that all ph'

# 3-Char Model

In [12]:
seq_length = 3
char_0 = [idx[i] for i in range(0, len(idx)-1-seq_length, seq_length)]
char_1 = [idx[i + 1] for i in range(0, len(idx)-1-seq_length, seq_length)]
char_2 = [idx[i + 2] for i in range(0, len(idx)-1-seq_length, seq_length)]
char_3 = [idx[i + 3] for i in range(0, len(idx)-1-seq_length, seq_length)]

In [13]:
x0 = np.array(char_0[:-2])
x1 = np.array(char_1[:-2])
x2 = np.array(char_2[:-2])
x3 = np.array(char_3[:-2])

In [14]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [15]:
n_fac = 42

In [16]:
inp0, emb0 = embedding_input('c0_dat', vocab_size, n_fac)
inp1, emb1 = embedding_input('c1_dat', vocab_size, n_fac)
inp2, emb2 = embedding_input('c2_dat', vocab_size, n_fac)

In [17]:
n_hidden = 256

In [18]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='tanh')
dense_out = Dense(vocab_size, activation='softmax')

In [19]:
# these embs are already linked to the inps from the embedding_input function
hidden = dense_in(emb0)
c2_dense = dense_in(emb1)
hidden = merge([dense_hidden(hidden), c2_dense])
c3_dense = dense_in(emb2)
hidden = merge([dense_hidden(hidden), c3_dense])
c4_out = dense_out(hidden)

In [20]:
model = Model([inp0, inp1, inp2], c4_out)

In [21]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
# model.optimizer.lr = 0.000001
# model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [22]:
# model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [23]:
model.optimizer.lr = 0.01

In [24]:
# model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [25]:
# model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [26]:
model.optimizer.lr = 0.000001

In [27]:
# model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [28]:
# model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [29]:
# model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [30]:
# model.fit([x0, x1, x2], x3, batch_size=64, nb_epoch=4)

In [31]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    return chars[i]

In [32]:
get_next('the')

'R'

In [33]:
get_next('thi')

'L'

In [34]:
get_next('hel')

'!'

In [35]:
get_next('Som')

'\x86'

In [36]:
get_next('Wha')

'!'

In [37]:
get_next('is ')

'Q'

In [38]:
get_next('s t')

'Z'

# Our First RNN

In [39]:
cs = 8

In [40]:
c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)] for n in range(cs)]

In [41]:
len(c_in_dat[0])

75112

In [42]:
len(c_in_dat), len(c_in_dat[0])

(8, 75112)

In [43]:
c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]

In [44]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [45]:
len(xs), len(xs[0])

(8, 75110)

In [46]:
y = np.stack(c_out_dat[:-2])

In [47]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [48]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [49]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [50]:
n_fac = 42

In [51]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [52]:
cs, vocab_size, n_fac

(8, 86, 42)

In [53]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [54]:
n_hidden = 256

In [55]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [56]:
hidden = dense_in(c_ins[0][1])

In [57]:
for i in range(1, cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

In [58]:
c_out = dense_out(hidden)

In [59]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [60]:
# model.fit(xs, y, batch_size=64, nb_epoch=12)

In [61]:
# model.fit(xs, y, batch_size=64, nb_epoch=12)

In [62]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [63]:
get_next('for thos')

'c'

In [64]:
get_next('part of ')

'N'

In [65]:
get_next('queens a')

'c'

# Our first RNN

In [66]:
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-cs, cs)] for n in range(cs)]

In [67]:
len(c_out_dat), len(c_out_dat[0])

(8, 75112)

In [68]:
[indices_char[i[0]] for i in c_in_dat]

['P', 'R', 'E', 'F', 'A', 'C', 'E', '\n']

In [69]:
[indices_char[i[0]] for i in c_out_dat]

['R', 'E', 'F', 'A', 'C', 'E', '\n', '\n']

In [70]:
[indices_char[i[1]] for i in c_in_dat]

['\n', '\n', 'S', 'U', 'P', 'P', 'O', 'S']

In [71]:
[indices_char[i[1]] for i in c_out_dat]

['\n', 'S', 'U', 'P', 'P', 'O', 'S', 'I']

In [72]:
ys = [np.stack(arr[:-2]) for arr in c_out_dat]

In [73]:
ys

[array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56]),
 array([ 1, 33,  2, ..., 71, 61, 54])]

In [74]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [75]:
[ys[n][:cs] for n in range(cs)]

[array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67]),
 array([ 1, 33,  2, 72, 67, 73,  2, 68])]

In [76]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [77]:
inp1 = Input(shape=(n_fac,), name = 'zeros')
hidden = dense_in(inp1)

In [78]:
outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = merge([c_dense, dense_hidden(hidden)], mode='sum')
    outs.append(dense_out(hidden)) 

In [79]:
len(outs)

8

In [80]:
model = Model(input=[inp1] + [c[0] for c in c_ins], output=outs)

In [80]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [81]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))

In [82]:
zeros

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [83]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12)

Epoch 1/12
75110/75110 [==============================] - 10s - loss: 20.1353 - dense_9_loss_1: 2.6985 - dense_9_loss_2: 2.5644 - dense_9_loss_3: 2.5171 - dense_9_loss_4: 2.4795 - dense_9_loss_5: 2.4763 - dense_9_loss_6: 2.4619 - dense_9_loss_7: 2.4734 - dense_9_loss_8: 2.4643    
Epoch 2/12
75110/75110 [==============================] - 10s - loss: 17.8798 - dense_9_loss_1: 2.5137 - dense_9_loss_2: 2.3520 - dense_9_loss_3: 2.2379 - dense_9_loss_4: 2.1745 - dense_9_loss_5: 2.1612 - dense_9_loss_6: 2.1443 - dense_9_loss_7: 2.1584 - dense_9_loss_8: 2.1377    
Epoch 3/12
75110/75110 [==============================] - 10s - loss: 17.3053 - dense_9_loss_1: 2.4970 - dense_9_loss_2: 2.3304 - dense_9_loss_3: 2.1813 - dense_9_loss_4: 2.0929 - dense_9_loss_5: 2.0644 - dense_9_loss_6: 2.0446 - dense_9_loss_7: 2.0565 - dense_9_loss_8: 2.0381    
Epoch 4/12
75110/75110 [==============================] - 10s - loss: 16.9486 - dense_9_loss_1: 2.4907 - dense_9_loss_2: 2.3236 - dense_9_loss_3: 2.1505 -

In [85]:
[zeros] + xs

[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [86]:
zeros.shape

(75110, 42)

In [87]:
len(xs), len(xs[0])

(8, 75110)

In [88]:
xs[0]

array([40,  1, 33, ..., 72, 71, 61])

In [89]:
zeros[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.])

In [90]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [81]:
# def get_nexts(inp):
#     # get character indices for input
#     idxs = [char_indices[c] for c in inp]
#     # turn each index into a 1-d array
#     arrs = [np.array([i]) for i in idxs]
#     # predict these arrays, outputting a softmax
#     p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
#     # return the character that maximizes the softmax probability
#     return [chars[np.argmax(o)] for o in p]
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array([i]) for i in idxs]
    p = model.predict([np.reshape(np.zeros(n_fac), newshape=(-1, n_fac))] + arrs)
    return [chars[np.argmax(o)] for o in p]

In [82]:
np.reshape(np.zeros(n_fac), newshape=(-1, n_fac)).shape

(1, 42)

In [83]:
get_nexts(' this is')

[']', '\xc3', 'E', 'E', 'E', 'E', 'E', 'E']

In [84]:
get_nexts(' part of')

[']', 'P', 'P', 'E', 'E', 'E', 'E', 'E']

# Returning Sequences

In [85]:
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-cs, cs)] for n in range(cs)]

# Our First Keras RNN

In [ ]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)y

In [ ]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=cs),
    SimpleRNN(n_hidden, activation='relu', inner_init='identity'),
    Dense(vocab_size, activation='softmax')
])

In [ ]:
c_ins = [embedding_input(name='c_' + str(i), n_in=vocab_size, n_out=n_fac) for i in range(cs)]

In [ ]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [ ]:
len(idx)

In [ ]:
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [ ]:
n_hidden

In [ ]:
# outs = []
# for i in range(cs):
#     hidden = merge([dense_in(c_ins[i][1]), dense_hidden(hidden)], mode='sum')
#     outs.append(dense_out(hidden)) 

outs = []

for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    # every layer now has an output
    outs.append(dense_out(hidden))

In [ ]:
mdl = Model([inp1] + [c[0] for c in c_ins], outs)

In [ ]:
len(c_ins)

In [ ]:
zs = np.tile(np.zeros(n_fac), (len(xs[0]), 1))

In [ ]:
x0 = [np.array([0])] * len(xs[0])

In [ ]:
zs.shape

In [ ]:
mdl.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
mdl.fit([zs]+xs, y, batch_size=64, nb_epoch=12)

# Sequence Model w/ Keras

In [86]:
n_hidden, n_fac, cs, vocab_size

(256, 42, 8, 86)

In [87]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=cs),
    SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

In [88]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 8, 42)         3612        embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 8, 256)        76544       embedding_4[0][0]                
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 8, 86)         22102       simplernn_1[0][0]                
Total params: 102,258
Trainable params: 102,258
Non-trainable params: 0
____________________________________________________________________________________________________


In [90]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [91]:
x_rnn = np.stack(xs, axis=1)

In [92]:
xs

[array([40,  1, 33, ..., 72, 71, 61]),
 array([42,  1, 38, ..., 73, 65, 58]),
 array([29, 43, 31, ..., 62, 57,  2]),
 array([30, 45,  2, ..., 54,  2, 62]),
 array([25, 40, 73, ..., 67, 54, 67]),
 array([27, 40, 61, ...,  2, 72, 57]),
 array([29, 39, 54, ..., 76,  2, 62]),
 array([ 1, 43, 73, ..., 68, 73, 56])]

In [93]:
x_rnn

array([[40, 42, 29, ..., 27, 29,  1],
       [ 1,  1, 43, ..., 40, 39, 43],
       [33, 38, 31, ..., 61, 54, 73],
       ..., 
       [72, 73, 62, ...,  2, 76, 68],
       [71, 65, 57, ..., 72,  2, 73],
       [61, 58,  2, ..., 57, 62, 56]])

In [96]:
y_rnn = np.expand_dims(np.stack(ys, axis=1), -1)

In [97]:
y_rnn

array([[[42],
        [29],
        [30],
        ..., 
        [29],
        [ 1],
        [ 1]],

       [[ 1],
        [43],
        [45],
        ..., 
        [39],
        [43],
        [33]],

       [[38],
        [31],
        [ 2],
        ..., 
        [54],
        [73],
        [ 2]],

       ..., 
       [[73],
        [62],
        [54],
        ..., 
        [76],
        [68],
        [71]],

       [[65],
        [57],
        [ 2],
        ..., 
        [ 2],
        [73],
        [61]],

       [[58],
        [ 2],
        [62],
        ..., 
        [62],
        [56],
        [54]]])

In [98]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 4s - loss: 2.4312     
Epoch 2/8
75110/75110 [==============================] - 4s - loss: 2.0016     
Epoch 3/8
75110/75110 [==============================] - 4s - loss: 1.8868     
Epoch 4/8
75110/75110 [==============================] - 4s - loss: 1.8259     
Epoch 5/8
75110/75110 [==============================] - 4s - loss: 1.7876     
Epoch 6/8
75110/75110 [==============================] - 4s - loss: 1.7601     
Epoch 7/8
75110/75110 [==============================] - 4s - loss: 1.7397     
Epoch 8/8
75110/75110 [==============================] - 4s - loss: 1.7233     


In [99]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8)

Epoch 1/8
75110/75110 [==============================] - 4s - loss: 1.7103     
Epoch 2/8
75110/75110 [==============================] - 4s - loss: 1.6994     
Epoch 3/8
75110/75110 [==============================] - 4s - loss: 1.6902     
Epoch 4/8
75110/75110 [==============================] - 4s - loss: 1.6820     
Epoch 5/8
75110/75110 [==============================] - 4s - loss: 1.6753     
Epoch 6/8
75110/75110 [==============================] - 4s - loss: 1.6681     
Epoch 7/8
75110/75110 [==============================] - 4s - loss: 1.6632     
Epoch 8/8
75110/75110 [==============================] - 4s - loss: 1.6582     


In [114]:
def get_next(inp):
    idx = np.array([char_indices[i] for i in inp])
    arr = idx[np.newaxis, :]
    pred = model.predict(arr)[0]
    return [indices_char[np.argmax(o)] for o in pred]

In [115]:
get_next('The thin')

['h', 'e', ' ', 's', 'r', 'e', 'n', 'k']

In [117]:
get_next(' this is')

['t', 'h', 'e', 's', ' ', 'a', 's', ' ']

In [120]:
model = Sequential([
   SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size), 
            activation='relu', inner_init='identity'),
   TimeDistributed(Dense(vocab_size, activation='softmax'))
])

In [123]:
oh_xs = np.stack([to_categorical(i, vocab_size) for i in xs], 1)

In [124]:
oh_ys = np.stack([to_categorical(i, vocab_size) for i in ys], 1)

In [125]:
oh_xs.shape

(75110, 8, 86)

In [126]:
oh_ys.shape

(75110, 8, 86)

In [127]:
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [128]:
model.fit(oh_xs, oh_ys, batch_size=64)

Epoch 1/10
75110/75110 [==============================] - 4s - loss: 2.4409     
Epoch 2/10
75110/75110 [==============================] - 4s - loss: 2.0385     
Epoch 3/10
75110/75110 [==============================] - 4s - loss: 1.9253     
Epoch 4/10
75110/75110 [==============================] - 4s - loss: 1.8611     
Epoch 5/10
75110/75110 [==============================] - 4s - loss: 1.8186     
Epoch 6/10
75110/75110 [==============================] - 4s - loss: 1.7873     
Epoch 7/10
75110/75110 [==============================] - 4s - loss: 1.7625     
Epoch 8/10
75110/75110 [==============================] - 4s - loss: 1.7444     
Epoch 9/10
75110/75110 [==============================] - 4s - loss: 1.7286     
Epoch 10/10
75110/75110 [==============================] - 4s - loss: 1.7156     


# Stateful Model with Keras

In [153]:
bs = 64

In [154]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs, 8)),
    BatchNormalization(),
    LSTM(n_hidden, return_sequences=True, stateful=True),
    TimeDistributed(Dense(n_hidden, activation='softmax'))
])

In [155]:
x_rnn.shape

(75110, 8)

In [156]:
mx = len(x_rnn)//bs * bs

In [157]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy')

In [158]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 17s - loss: 2.3300    
Epoch 2/4
75072/75072 [==============================] - 17s - loss: 2.0633    
Epoch 3/4
75072/75072 [==============================] - 17s - loss: 1.9773    
Epoch 4/4
75072/75072 [==============================] - 17s - loss: 1.9269    


In [136]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 17s - loss: 1.8778    
Epoch 2/4
75072/75072 [==============================] - 17s - loss: 1.8512    
Epoch 3/4
75072/75072 [==============================] - 16s - loss: 1.8295    
Epoch 4/4
75072/75072 [==============================] - 16s - loss: 1.8113    


In [137]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False)

Epoch 1/4
75072/75072 [==============================] - 17s - loss: 1.7954    
Epoch 2/4
75072/75072 [==============================] - 16s - loss: 1.7814    
Epoch 3/4
75072/75072 [==============================] - 17s - loss: 1.7688    
Epoch 4/4
75072/75072 [==============================] - 16s - loss: 1.7571    


In [140]:
def get_next_stateful(inp):
    idx = [char_indices[i] for i in inp]
    arr = np.array(idx)
    pred = model.predict(arr)[0]
    return [indices_char[np.argmax(o)] for o in pred]

In [142]:
x_rnn.shape

(75110, 8)

In [159]:
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        print(x)
        raise SystemExit
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [160]:
print_example()

[[58 73 61 62 56 72  2 62 72  2 54  2 55 54 72 62 56  2 59 68 74 67 57 54 73 62 68 67  2 68 59  2 54
  65 65  2 73 61 54 73]]


SystemExit: 